Imports and User parameters


In [127]:
import pandas as pd
import numpy as np
import math

df = pd.read_excel("../data/dados-2025.xlsx", sheet_name="Planilha1")
row = df.loc[df["Nome"].str.contains("Arthur Rabello Oliveira")].iloc[0]

iD, m, c, f, N, M = (
    int(row["ID"]),
    float(row["m"]),
    float(row["c"]),
    float(row["f"]),
    int(float(row["N"])),
    int(float(row["M"])),
)

print(f"id = {iD}, m = {m}, c = {c}, f = {f}, N = {N}, M = {M}")


id = 2, m = 5.5, c = 0.02, f = 0.35, N = 425, M = 3000


Problema 1 a)

In [128]:
# ------------------------
# Exercise parameters
# ------------------------
arrival_rate      = 1 / 10  # arrival rate (1 customer every 10 min ⇒ λ = 0.1 min⁻¹)
n_customers = 10_000        # simulation size

fast_fraction = 0.35        # fraction of “fast” customers
slope = 0.02                # slope of the linear pdf (2–6 min)
mean_exp = 5.5              # mean (min) of the exponential service time for the others

# ------------------------
# Utility function: samples service times for fast customers
# pdf:  f(x) = 0.25 + slope·(x – 4)  for  2 < x < 6
# ------------------------
def sample_linear_en(size, slope=slope):
    a = 0.25 - 4 * slope         # f(x) = a + b·x  (b = slope)
    b = slope
    A = b / 2                    # coefficient of x² in the CDF
    B = a
    u = np.random.rand(size)     # uniform samples (0, 1)

    # Solve A·x² + B·x – (2a + 2b) – u = 0  (positive root → x ∈ (2, 6))
    C = -2 * a - 2 * b - u
    x = (-B + np.sqrt(B ** 2 - 4 * A * C)) / (2 * A)
    return x

# ------------------------
# 1. Generate arrivals
# ------------------------
rng = np.random.default_rng()
inter_arrival_times = rng.exponential(scale=1 / arrival_rate, size=n_customers)  # scale = 10 min
arrivals = np.cumsum(inter_arrival_times)

# ------------------------
# 2. Generate service times
# ------------------------
is_fast = rng.random(n_customers) < fast_fraction

service_times = np.empty(n_customers)
service_times[is_fast]  = sample_linear_en(is_fast.sum())
service_times[~is_fast] = rng.exponential(scale=mean_exp, size=(~is_fast).sum())

# ------------------------
# 3. Simulate the queue (M/G/1) and measure waits/idleness
# ------------------------
start_times   = np.empty(n_customers)
end_times     = np.empty(n_customers)
idle_time = 0.0

# customer 0
start_times[0] = arrivals[0]
end_times[0]   = start_times[0] + service_times[0]

# remaining customers
for i in range(1, n_customers):
    if arrivals[i] > end_times[i - 1]:
        idle_time += arrivals[i] - end_times[i - 1]          # server was idle
    start_times[i] = max(arrivals[i], end_times[i - 1])
    end_times[i]   = start_times[i] + service_times[i]

wait_times = start_times - arrivals

# ------------------------
# 4. Requested statistics
# ------------------------
mean_service  = service_times.mean()
std_service   = service_times.std(ddof=1)
mean_wait     = wait_times.mean()
std_wait      = wait_times.std(ddof=1)
p_wait3       = np.mean(wait_times > 3)            # P(wait > 3 min)
idle_fraction = idle_time / end_times[-1]

print(f"Service duration:  mean = {mean_service:.3f} min,  std = {std_service:.3f} min")
print(f"Waiting time:      mean = {mean_wait:.3f} min,  std = {std_wait:.3f} min")
print(f"P(wait > 3 min) = {p_wait3:.3%}")
print(f"Fraction of server idle time = {idle_fraction:.3%}")


Service duration:  mean = 5.057 min,  std = 4.644 min
Waiting time:      mean = 4.803 min,  std = 8.457 min
P(wait > 3 min) = 36.670%
Fraction of server idle time = 48.830%


Problem 2 (a) and (c)

In [129]:
# ------------------------
# Problem parameters
# ------------------------   
n_rep = 1000     # number of simulation repetitions
rng   = np.random.default_rng()

def simulate_album(N):
    """Simulates the total number of stickers bought until completing 1 album."""
    missing = set(range(N))
    purchases = 0
    while missing:
        s = rng.integers(N)   # drawn sticker
        purchases += 1
        missing.discard(s)
    return purchases

totals = np.array([simulate_album(N) for _ in range(n_rep)])

mean_sim = totals.mean()
std_sim  = totals.std(ddof=1)
p_greater_M = np.mean(totals > M)

print(f"Item (a) — simulation 1 album (n={n_rep}):")
print(f"  mean = {mean_sim:.2f}")
print(f"  standard deviation = {std_sim:.2f}")

print(f"\nItem (c) — P(total > {M}) ≈ {p_greater_M:.3%}")


Item (a) — simulation 1 album (n=1000):
  mean = 2829.78
  standard deviation = 553.49

Item (c) — P(total > 3000) ≈ 31.100%


Problem 2 item (b)

In [130]:
# Probability of “success” (drawing a new sticker) when there are j missing
p = np.arange(N, 0, -1) / N          # p_j  for j = N, …, 1

# Each step has geometric(p_j) distribution (support 1,2,…)
esp = (1 / p).sum()         
var = ((1 - p)/p**2).sum()          
desv = math.sqrt(var)

print("Item (b) — analytical values for 1 album:")
print(f"  exact mean        = {esp:.2f}")
print(f"  exact standard deviation = {desv:.2f}")

print("\nComparison with the simulation (Previous Cell):")
print(f"  relative difference in the mean = {(mean_sim/esp - 1):.3%}")

Item (b) — analytical values for 1 album:
  exact mean        = 2817.95
  exact standard deviation = 542.10

Comparison with the simulation (Previous Cell):
  relative difference in the mean = 0.420%


Problem 2 item (d)

In [131]:
def simulate_two_albums(N):
    """Simulates purchases until TWO albums are completed, allowing exchanges."""
    missing_A = set(range(N))
    missing_B = set(range(N))
    purchases = 0

    while missing_A or missing_B:
        s = rng.integers(N)
        purchases += 1
        # Priority: give the sticker to whoever doesn't have it yet
        if s in missing_A:
            missing_A.remove(s)
        elif s in missing_B:
            missing_B.remove(s)
        # otherwise both already had it ⇒ extra copy with no use
    return purchases

totals_2 = np.array([simulate_two_albums(N) for _ in range(n_rep)])
mean_2  = totals_2.mean()

print("Item (d) — two people with exchanges:")
print(f"  mean purchases to complete BOTH albums = {mean_2:.2f}")
print(f"  ratio compared to 1 album (Cell 2)     = {mean_2 / mean_sim:.3f}")

Item (d) — two people with exchanges:
  mean purchases to complete BOTH albums = 3802.36
  ratio compared to 1 album (Cell 2)     = 1.344


Simple Analysis

In [132]:
print(f"1 album                             : {mean_sim:7.0f}")
print(f"2 albums, WITHOUT exchange (2x)     : {2*mean_sim:7.0f}")
print(f"2 albums, WITH exchange             : {mean_2:7.0f}")
print(f"Absolute savings                    : {2*mean_sim - mean_2:7.0f}")
print(f"Relative savings                    : {(1-mean_2/(2*mean_sim)):.1%}")


1 album                             :    2830
2 albums, WITHOUT exchange (2x)     :    5660
2 albums, WITH exchange             :    3802
Absolute savings                    :    1857
Relative savings                    : 32.8%
